### 環境設定

In [14]:
import os
from dotenv import load_dotenv
import settings
from glob import glob
from pdf_reader import pdf_reader_run



%load_ext autoreload
%autoreload 2

load_dotenv()

# Access the environment variables
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

## LLM
openai_api_key = os.getenv('OPENAI_API_KEY')
PROJECT_NAME = settings.LANGCHAIN_PROJECT

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### PDFの読み込みと処理

In [9]:
file_paths = glob(settings.pdf_file_path+"/*")
docs_list = pdf_reader_run(file_paths)

成功 (PDFPlumber): ./documents\1.pdf
成功 (PDFPlumber): ./documents\10.pdf
成功 (PDFPlumber): ./documents\11.pdf
成功 (PDFPlumber): ./documents\12.pdf
成功 (PDFPlumber): ./documents\13.pdf


Data-loss while decompressing corrupted data


成功 (PDFPlumber): ./documents\14.pdf
PDFPlumberが失敗: ./documents\15.pdf - エラー: Invalid dictionary construct: [/'op', True, /'Type', /'ExtGState', /'AIS', /b'fal', /b'se', /'BM', /'Normal', /'CA', 1, /'OP', True, /'OPM', 1, /'SA', True, /'SMask', /'None', /'ca', 1]
成功 (UnstructuredPDF-fast): ./documents\15.pdf
成功 (PDFPlumber): ./documents\16.pdf
PDFPlumberが失敗: ./documents\17.pdf - エラー: Invalid dictionary construct: [/'CS', /'DeviceGray', /'I', /b'fal', /b'se', /'K', False, /'S', /'Transparency', /'Type', /'Group']
成功 (UnstructuredPDF-fast): ./documents\17.pdf
成功 (PDFPlumber): ./documents\18.pdf


Data-loss while decompressing corrupted data


成功 (PDFPlumber): ./documents\19.pdf
成功 (PDFPlumber): ./documents\2.pdf
成功 (PDFPlumber): ./documents\3.pdf
成功 (PDFPlumber): ./documents\4.pdf
成功 (PDFPlumber): ./documents\5.pdf
成功 (PDFPlumber): ./documents\6.pdf
成功 (PDFPlumber): ./documents\7.pdf
成功 (PDFPlumber): ./documents\8.pdf
PDFPlumberが失敗: ./documents\9.pdf - エラー: Invalid dictionary construct: [/'CS', <PDFObjRef:5006>, /'I', /b'tru', /b'e', /'K', False, /'S', /'Transparency', /'Type', /'Group']
成功 (UnstructuredPDF-fast): ./documents\9.pdf

=== 処理結果サマリー ===
総ファイル数: 19
成功したファイル数: 19
失敗したファイル数: 0


### text split

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=settings.CHUNK_SIZE,
    chunk_overlap=settings.CHUNK_OVERLAP
)
splits = text_splitter.split_documents(docs_list)

In [17]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",  # 中国語/日本語に強い
    encode_kwargs={'normalize_embeddings': True}
)

In [ ]:
faiss_vectorstore = FAISS.from_documents(
                        documents=splits,
                        embedding=embeddings
                    )

In [ ]:
bge_retriever = faiss_vectorstore.as_retriever()

In [ ]:
# 保存
faiss_vectorstore.save_local(".vectorstore/faiss_index")